In [ ]:
%matplotlib inline

%load_ext autoreload
%autoreload 2

## Stdlib
from collections import defaultdict, namedtuple
from datetime import datetime
import math
import os
import sys
import tempfile

## Non-std libs
import matplotlib.pyplot as plt
import pandas as pd

## Local modules
from scn_rrd import config, librenms_meta_utils, librenms_rrd_utils, rrd_utils, plot_utils

### Data rate consumption

In [ ]:
portGroups = [
    'epc-backhaul-interface',
    'backhaul-interface',
]

meta = librenms_meta_utils.read_meta()
meta = meta[
    ( meta['location'].isin(config.PHYS_LOCS) ) &
    ( meta['port_group_name'].isin(portGroups) )
]

In [ ]:
# For each device, there should be one interface/port that's tagged as "backhaul interface" on LibreNMS.
len(meta['hostname'].unique()) == len(meta)

In [ ]:
NMS_HOST_IP = config.DOTENV_ENTRIES['NMS_HOST_IP']
physLoc_to_consumeDf = dict()
for (_, row) in meta.iterrows():
    rrd_filename = librenms_rrd_utils.format_port_rrd_filename(row['port_id'])
    rrd_filepath = librenms_rrd_utils.format_rrd_filepath(row['hostname'], rrd_filename)
    df = rrd_utils.read_rrd(NMS_HOST_IP, rrd_filepath, '-3month')

    physLoc = row['location']

    physLoc_to_consumeDf[physLoc] = df

In [ ]:
(fig, ax) = plt.subplots(figsize=(12, 6))
for (physLoc, consumeDf) in physLoc_to_consumeDf.items():
    dt_sec = (consumeDf['time'][1] - consumeDf['time'][0]).seconds
    
    # Raw avg bytes per step --> tot bytes per step --> tot bytes per day --> tot GB per day.
    factor = dt_sec / pow(2, 30)
    aggr_df = (
        consumeDf[['time', 'INOCTETS', 'OUTOCTETS']]
        .groupby(by=pd.Grouper(key='time', freq='D'))
        .sum()
        * factor
    )
    
    color = plot_utils.LOC_TO_COLOR[physLoc]
    aggr_df['INOCTETS'].plot(ax=ax, label=f"{physLoc} downloads", color=color, linestyle='solid')
    aggr_df['OUTOCTETS'].plot(ax=ax, label=f"{physLoc} uploads", color=color, linestyle='dashed')
ax.set_title('Daily consumption')
ax.set_ylabel('Gigabyte')
ax.set_xlabel('Day')
ax.legend()
None # Hide stdout output of the above line

In [ ]:
(fig, ax) = plt.subplots(figsize=(12, 6))
for (physLoc, consumeDf) in physLoc_to_consumeDf.items():
    dt_sec = (consumeDf['time'][1] - consumeDf['time'][0]).seconds
    days = ( max(consumeDf['time']) - min(consumeDf['time']) ).days

    # Raw avg bytes per step --> tot bytes per step --> tot bytes per hour-of-day
    #   --> tot GB per hour-of-day --> tot GB per hour-of-day per avg day
    factor = dt_sec / pow(2, 30) / days
    aggr_df = (
        consumeDf[['time', 'INOCTETS', 'OUTOCTETS']]
        .groupby(consumeDf['time'].dt.hour)
        .sum(numeric_only=True)
        * factor
    )

    color = plot_utils.LOC_TO_COLOR[physLoc]
    aggr_df['INOCTETS'].plot(ax=ax, label=f"{physLoc} downloads", color=color, linestyle='solid')
    aggr_df['OUTOCTETS'].plot(ax=ax, label=f"{physLoc} uploads", color=color, linestyle='dashed')
ax.set_title('Average hourly consumption')
ax.set_ylabel('Gigabyte')
ax.set_xlabel('Hour')
ax.legend()
None # Hide stdout output of the above line

### Data rate capacity (aka speedtest)

In [ ]:
physLoc_to_capDf = dict()
for (ip, physLoc) in config.MONITOR_DEVICES.items():
    down_df = rrd_utils.read_rrd(ip, 'down_rate.rrd', '-3month')
    up_df   = rrd_utils.read_rrd(ip, 'up_rate.rrd',   '-3month')

    df = down_df.merge(up_df, on='time', how='outer')

    physLoc_to_capDf[physLoc] = df

In [ ]:
(fig, ax) = plt.subplots(figsize=(12, 6))
for phys_loc in config.MONITOR_DEVICES.values():
    color = plot_utils.LOC_TO_COLOR[phys_loc]
    
    capDf = physLoc_to_capDf[phys_loc]
    
    ax.plot(capDf['time'], capDf['down_rate'], label=f"{phys_loc} download", color=color, linestyle='solid')
    ax.plot(capDf['time'], capDf['up_rate'], label=f"{phys_loc} upload", color=color, linestyle='dashed')
ax.set_title('Capacity')
ax.set_ylabel('Mbps')
ax.legend()
None # Hide stdout output of the above line

### Utilization == Consumption / Capacity

In [ ]:
def utilization(consume_df: pd.DataFrame, capacity_df: pd.DataFrame, factor: float) -> pd.DataFrame:
    '''
    @arg consume_df and capacity_df:
        Each DF should have two columns,
        1st column containing time, and
        2nd column containing quantity (float).
    @return:
        1st column contains time, unmodified from @arg consume_df.
        2nd column contains (consumed quantity / capacity quantity),
            where capacity quantity comes from the last measurement at or before the time.
    '''
    cap_i = 0
    cap_n = len(capacity_df)
    cap_quant_prev = None
    
    utilizn_rows = []
    
    for (_, (consu_time, consu_quant)) in consume_df.iterrows():
        if pd.isna(consu_quant):
            continue
        
        while cap_i < cap_n and capacity_df.iloc[cap_i][0] <= consu_time:
            _cap_quant = capacity_df.iloc[cap_i][1]
            if not pd.isna(_cap_quant):
                cap_quant_prev = _cap_quant
            cap_i += 1

        if pd.isna(cap_quant_prev):
            # No capacity was recorded before the current consumption timestamp.
            continue

        utilizn = consu_quant / cap_quant_prev * factor
        utilizn_row = (consu_time, utilizn)
        utilizn_rows.append(utilizn_row)

    return pd.DataFrame(utilizn_rows, columns=['time', 'utilization'])

In [ ]:
mon_ct = len(config.MONITOR_DEVICES)
(fig, axez) = plt.subplots(nrows=mon_ct, figsize=(12, 3 * mon_ct), constrained_layout=True)
for (ax, phys_loc) in zip(axez, config.MONITOR_DEVICES.values()):
    logi_loc = config.PHYS_LOC_TO_LOGICAL_LOC[phys_loc]
    
    ## Get consumption at the monitoring device or at our LTE location.
    consumeDf = physLoc_to_consumeDf.get(phys_loc, physLoc_to_consumeDf[logi_loc])
    ## Get capacity at the monitoring device.
    capDf = physLoc_to_capDf[phys_loc]
    
    # Scale both numerator and denominator to bitsPerSec. Then scale to percentage.
    factor = (1/8) / pow(2,20) * 100
    
    color = plot_utils.LOC_TO_COLOR[phys_loc]

    utilzn_df = utilization(consumeDf[['time', 'INOCTETS']], capDf[['time', 'down_rate']], factor)
    ax.plot(utilzn_df['time'], utilzn_df['utilization'], label=f"{phys_loc} download", color=color, linestyle='solid')
    
    utilizn_df = utilization(consumeDf[['time', 'OUTOCTETS']], capDf[['time', 'up_rate']], factor)
    ax.plot(utilzn_df['time'], utilzn_df['utilization'], label=f"{phys_loc} upload", color=color, linestyle='dashed')
    
    ax.set_ylabel('Percent')
    ax.legend()
fig.suptitle('Utilization')
None # Hide stdout output of the above line